<title>Data understanding</title>

<h1>Data understanding</h1>

In [1]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [2]:

import pyspark
from pyspark.sql.functions import when, col, count, rand, sum, concat, lit, array,size, concat_ws
from pyspark.ml.fpm import FPGrowth, PrefixSpan
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Itteration 4').getOrCreate()

<h2>Import datasets</h2>

In [4]:
app_raw = spark.read.csv('d:/University/Postgrad/2023 Sem 2/INFOSYS 722/Assignments/Data used/application_data.csv', header=True, inferSchema=True)
prev_raw = spark.read.csv("d:/University/Postgrad/2023 Sem 2/INFOSYS 722/Assignments/Data used/bureau.csv", header=True, inferSchema=True)

In [5]:
app_raw.show()

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+-------------------+-------------------+-------------------+--------------+----------------+---------------------------+------------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+------------

In [6]:
prev_raw.show()

+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|    CREDIT_TYPE|DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|    215354|     5714462|       Closed|     currency 1|       -497|                 0|             -153.0|           -153

<h2>Data prerperation</h2>

In [7]:
app_df = app_raw[['SK_ID_CURR','NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','CNT_CHILDREN','AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE','NAME_TYPE_SUITE','NAME_INCOME_TYPE',
'NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE','DAYS_BIRTH','DAYS_EMPLOYED','DAYS_REGISTRATION','DAYS_ID_PUBLISH','OWN_CAR_AGE','OCCUPATION_TYPE','CNT_FAM_MEMBERS','WEEKDAY_APPR_PROCESS_START',
'HOUR_APPR_PROCESS_START','ORGANIZATION_TYPE','AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR']]
app_df.show()

+----------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+-----------------+---------------+-----------+---------------+---------------+--------------------------+-----------------------+--------------------+--------------------------+-------------------------+--------------------------+-------------------------+-------------------------+--------------------------+
|SK_ID_CURR|NAME_CONTRACT_TYPE|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|AMT_CREDIT|AMT_ANNUITY|AMT_GOODS_PRICE|NAME_TYPE_SUITE|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|DAYS_REGISTRATION|DAYS_ID_PUBLISH|OWN_CAR_AGE|OCCUPATION_TYPE|CNT_FAM_MEMBERS|WEEKDAY_APPR_PROCESS_START|HOUR_APPR_PROCESS_START|   ORGANIZATION_TYPE|AMT_REQ_CREDIT

<h3>Find completness</h3>

In [8]:
def get_completeness(df_given):
    # Calculate completeness for each column
    completeness_data = []

    for column in df_given.columns:
        non_null_count = df_given.filter(col(column).isNotNull()).count()
        total_count = df_given.count()
        completeness = (non_null_count / total_count) * 100
        completeness_data.append((column, completeness))

    # Create a DataFrame to store the completeness results
    return spark.createDataFrame(completeness_data, ["Column", "Completeness"])


In [9]:
app_df_completness = get_completeness(app_df)
prev_raw_completness = get_completeness(prev_raw)

In [ ]:
app_df_completness.show(50)
prev_raw_completness.show()

<h3>Fill incomplete columns</h3>

In [16]:
fill_values = {"NAME_TYPE_SUITE" : "None",
                "OCCUPATION_TYPE" : "Unknown",
                'OWN_CAR_AGE' : 0,'CNT_FAM_MEMBERS' : 0,
                'AMT_ANNUITY' : 0,'AMT_GOODS_PRICE' : 0,
                'AMT_REQ_CREDIT_BUREAU_HOUR' : 0,
                'AMT_REQ_CREDIT_BUREAU_DAY' : 0,
                'AMT_REQ_CREDIT_BUREAU_WEEK' : 0,
                'AMT_REQ_CREDIT_BUREAU_MON' : 0,
                'AMT_REQ_CREDIT_BUREAU_QRT' : 0,
                'AMT_REQ_CREDIT_BUREAU_YEAR' : 0}
app_df_complete = app_df.fillna(fill_values)
completness_check = get_completeness(app_df_complete)

In [17]:
completness_check.show(50)

Py4JJavaError: An error occurred while calling o434.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 486.0 failed 1 times, most recent failure: Lost task 0.0 in stage 486.0 (TID 1448) (172.31.129.41 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\blake\anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 1100, in main
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 9) than that in driver 3.11, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:784)
	at org.apache.spark.api.python.PythonRunne

In [ ]:
app_df = app_df_complete

<h3>Day values</h3>

In [ ]:
columns_to_multiply = ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH']

for col_name in columns_to_multiply:
    app_df = app_df.withColumn(col_name, when(col(col_name) < 0, col(col_name) * -1).otherwise(col(col_name)))
app_df.show()

<h3>Renaming columns</h3>

In [ ]:
app_df_renamed= app_df.withColumnRenamed('NAME_CONTRACT_TYPE', 'CONTRACT_TYPE') \
    .withColumnRenamed('CODE_GENDER', 'GENDER') \
    .withColumnRenamed('FLAG_OWN_CAR', 'OWN_CAR') \
    .withColumnRenamed('FLAG_OWN_REALTY', 'OWN_REALTY') \
    .withColumnRenamed('CNT_CHILDREN', 'NUMBER_CHILDREN') \
    .withColumnRenamed('AMT_INCOME_TOTAL', 'AMT_INCOME_TOTAL') \
    .withColumnRenamed('AMT_CREDIT', 'AMT_CREDIT') \
    .withColumnRenamed('AMT_ANNUITY', 'AMT_ANNUITY') \
    .withColumnRenamed('AMT_GOODS_PRICE', 'AMT_GOODS_PRICE') \
    .withColumnRenamed('NAME_TYPE_SUITE', 'REFERENCE_TYPE') \
    .withColumnRenamed('NAME_INCOME_TYPE', 'INCOME_TYPE') \
    .withColumnRenamed('NAME_EDUCATION_TYPE', 'EDUCATION_TYPE') \
    .withColumnRenamed('NAME_FAMILY_STATUS', 'FAMILY_STATUS') \
    .withColumnRenamed('NAME_HOUSING_TYPE', 'HOUSING_TYPE') \
    .withColumnRenamed('REGION_POPULATION_RELATIVE', 'REGION_POPULATION_RELATIVE') \
    .withColumnRenamed('DAYS_BIRTH', 'DAYS_BIRTH') \
    .withColumnRenamed('DAYS_EMPLOYED', 'DAYS_EMPLOYED') \
    .withColumnRenamed('DAYS_REGISTRATION', 'DAYS_REGISTRATION') \
    .withColumnRenamed('DAYS_ID_PUBLISH', 'DAYS_ID_PUBLISH') \
    .withColumnRenamed('OWN_CAR_AGE', 'CAR_AGE') \
    .withColumnRenamed('OCCUPATION_TYPE', 'OCCUPATION_TYPE') \
    .withColumnRenamed('CNT_FAM_MEMBERS', 'NUMBER_OF_FAM_MEMBERS') \
    .withColumnRenamed('WEEKDAY_APPR_PROCESS_START', 'WEEKDAY_PROCESS_START') \
    .withColumnRenamed('HOUR_APPR_PROCESS_START', 'HOUR_PROCESS_START') \
    .withColumnRenamed('ORGANIZATION_TYPE', 'ORGANIZATION_TYPE') \
    .withColumnRenamed('AMT_REQ_CREDIT_BUREAU_HOUR', 'CREDIT_INQUIRY_HOUR_PRIOR') \
    .withColumnRenamed('AMT_REQ_CREDIT_BUREAU_DAY', 'CREDIT_INQUIRY_DAY_PRIOR') \
    .withColumnRenamed('AMT_REQ_CREDIT_BUREAU_WEEK', 'CREDIT_INQUIRY_WEEK_PRIOR') \
    .withColumnRenamed('AMT_REQ_CREDIT_BUREAU_MON', 'CREDIT_INQUIRY_MONTH_PRIOR') \
    .withColumnRenamed('AMT_REQ_CREDIT_BUREAU_YEAR', 'CREDIT_INQUIRY_YEAR_PRIOR') \
    .withColumnRenamed('AMT_REQ_CREDIT_BUREAU_QRT', 'CREDIT_INQUIRY_QTR_PRIOR') 

In [ ]:
app_df_renamed.show()

<h3>Make year values</h3>

In [ ]:
columns_to_multiply = ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH']

for col_name in columns_to_multiply:
    app_df = app_df.withColumn(col_name+"_IN_YEARS", col(col_name)/365)

In [ ]:
app_df.show()

<h2>Data intergration</h2>

In [ ]:
#Aggegate the dataset
grouped = prev_raw.groupBy("SK_ID_CURR")
prev_df = grouped.agg(count("*").alias("count"))

In [ ]:
#Merge datasets on SK_ID_CURR
merged_df = app_df.join(prev_df, on="SK_ID_CURR", how="inner")

<h2>Split training set</h2>

In [ ]:
#Split merged data frame into training and testing set by a ratio of 70% : 30%
train_ratio = 0.7
test_ratio = 1 - train_ratio
train_df, test_df = merged_df.randomSplit([train_ratio, test_ratio], seed=42)

print("Training Set:")
train_df.show()

print("Testing Set:")
test_df.show()

<h2>Data mining</h2>

<h3>FP Growth</h3>

In [ ]:
columns_to_combine=['SK_ID_CURR','NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','CNT_CHILDREN','AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE','NAME_TYPE_SUITE','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','DAYS_BIRTH','DAYS_EMPLOYED','DAYS_REGISTRATION','DAYS_ID_PUBLISH','OWN_CAR_AGE','OCCUPATION_TYPE','CNT_FAM_MEMBERS','WEEKDAY_APPR_PROCESS_START','HOUR_APPR_PROCESS_START','ORGANIZATION_TYPE','AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR','DAYS_BIRTH_IN_YEARS','DAYS_EMPLOYED_IN_YEARS','DAYS_REGISTRATION_IN_YEARS','DAYS_ID_PUBLISH_IN_YEARS','count']

df_with_list = merged_df.withColumn( "transaction",
    array(*[concat(lit(column + ': '), col(column)) for column in columns_to_combine])
)

df_with_list = df_with_list[['SK_ID_CURR', 'transaction']]

df_with_list.show(truncate=False)


In [ ]:
fp_growth = FPGrowth(itemsCol="transaction", minSupport=0.4, minConfidence=0.7)

model = fp_growth.fit(df_with_list)

In [ ]:
# Display frequent itemsets
print("Frequent Itemsets:")
model.freqItemsets.show(truncate=True)

# Display association rules
print("\nAssociation Rules:")
rules = model.associationRules
model.associationRules.show(50, truncate=False)

<h3>Visualise FP-Growth</h3>

In [ ]:
# Calculate the number of items in antecedents and filter
rules = rules.withColumn('lhs_items', size(col('antecedent')))
rules_filtered = rules.filter(rules['lhs_items'] > 1.4)

# Sort by 'lift' in descending order
sorted_rules = rules_filtered.orderBy(col('lift').desc())

# Show the top results
sorted_rules.show(50)

# Replace frozen sets with strings
rules_replaced = rules_filtered.withColumn('antecedent_', concat_ws(',', 'antecedent'))
rules_replaced = rules_replaced.withColumn('consequent_', concat_ws(',', 'consequent'))

In [ ]:
import pandas as pd
rules_df = rules_replaced.toPandas()
rules_df = rules_df[rules_df['lift']>1.1]
pivot = rules_df[rules_df['lhs_items']>1.4].pivot(index = 'antecedent_', 
                    columns = 'consequent_', values= 'lift')

# rules_df.head()
# Generate a heatmap with annotations on
sns.heatmap(pivot, annot=True)
plt.yticks(rotation=0)
plt.xticks(rotation=90)
plt.grid()
plt.show()

In [ ]:
# Find the statistics
max_support = rules_filtered.agg({"support": "max"}).collect()[0][0]
min_support = rules_filtered.agg({"support": "min"}).collect()[0][0]
max_confidence = rules_filtered.agg({"confidence": "max"}).collect()[0][0]
min_confidence = rules_filtered.agg({"confidence": "min"}).collect()[0][0]
max_lift = rules_filtered.agg({"lift": "max"}).collect()[0][0]
min_lift = rules_filtered.agg({"lift": "min"}).collect()[0][0]

print(f"Maximum Support: {max_support:.4f}")
print(f"Minimum Support: {min_support:.4f}")
print(f"Maximum Confidence: {max_confidence:.4f}")
print(f"Minimum Confidence: {min_confidence:.4f}")
print(f"Maximum Lift: {max_lift:.4f}")
print(f"Minimum Lift: {min_lift:.4f}")

<h3>K - means</h3>

In [ ]:
# Load your data into a DataFrame
data = app_df

# Identify non-numeric columns
non_numeric_cols = [col for col, dtype in data.dtypes if dtype == 'string']

# Create stages for one-hot encoding
stages = []

for col in non_numeric_cols:
    indexer = StringIndexer(inputCol=col, outputCol=col + "_index")
    encoder = OneHotEncoder(inputCol=col + "_index", outputCol=col + "_encoded")
    stages += [indexer, encoder]

# Assemble feature vectors
feature_cols = [col + "_encoded" for col in non_numeric_cols] + [col for col in data.columns if col not in non_numeric_cols]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
stages += [assembler]

# Create a pipeline
pipeline = Pipeline(stages=stages)
model = pipeline.fit(data)
data_transformed = model.transform(data)

# Select the relevant columns
selected_columns = ["features"] + [col for col in data.columns if col not in non_numeric_cols]
data_transformed = data_transformed.select(selected_columns)

# Show the transformed DataFrame
data_transformed.show(truncate=False)



In [ ]:
#Find silhouette score
k_values = [2, 3, 4, 5]  # Test different values of K
for k in k_values:
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(data_transformed)

    # Get the cluster assignments
    predictions = model.transform(data_transformed)

    # Evaluate the clustering using the Silhouette score
    evaluator = ClusteringEvaluator()
    silhouette_score = evaluator.evaluate(predictions)
    print(f"For K={k}, Silhouette Score: {silhouette_score}")

In [ ]:
#Actual K-Means program
kmeans = KMeans(featuresCol='features',k=2) 
model = kmeans.fit(data_transformed) 
predictions = model.transform(data_transformed)

In [ ]:
#Show cluster diffrences
grouped_data = predictions.groupBy("prediction")

# Define the attributes you want to aggregate
attributes_to_aggregate = [
    "CNT_CHILDREN",
    "AMT_INCOME_TOTAL",
    "AMT_CREDIT",
    "AMT_ANNUITY",
    "AMT_GOODS_PRICE",
    "Age",
    "Years_EMPLOYED",
    "Years_REGISTRATION",
    "Years_ID_PUBLISH",
    "OWN_CAR_AGE",
    "CNT_FAM_MEMBERS",
    "HOUR_APPR_PROCESS_START",
    "AMT_REQ_CREDIT_BUREAU_HOUR",
    "AMT_REQ_CREDIT_BUREAU_DAY",
    "AMT_REQ_CREDIT_BUREAU_WEEK",
    "AMT_REQ_CREDIT_BUREAU_MON",
    "AMT_REQ_CREDIT_BUREAU_QRT",
    "AMT_REQ_CREDIT_BUREAU_YEAR"
]

# Create aggregation expressions for each attribute
agg_exprs = {attr: "mean" for attr in attributes_to_aggregate}

# Calculate the mean of specified attributes within each cluster
cluster_means = grouped_data.agg(agg_exprs)

# Show the results
cluster_means.show()